<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/french_wow_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

In [2]:
!wget http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
!mkdir /usr/local/lib/python3.7/dist-packages/data
!mkdir /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia
!cp wizard_of_wikipedia.tgz /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/
!tar -zxvf /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/wizard_of_wikipedia.tgz -C /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/

--2022-06-09 09:43:52--  http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Resolving parl.ai (parl.ai)... 108.156.107.44, 108.156.107.71, 108.156.107.51, ...
Connecting to parl.ai (parl.ai)|108.156.107.44|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz [following]
--2022-06-09 09:43:52--  https://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Connecting to parl.ai (parl.ai)|108.156.107.44|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://dl.fbaipublicfiles.com/parlai/wizard_of_wikipedia/wizard_of_wikipedia.tgz [following]
--2022-06-09 09:43:52--  https://dl.fbaipublicfiles.com/parlai/wizard_of_wikipedia/wizard_of_wikipedia.tgz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|1

In [ ]:
!parlai display_data --task wizard_of_wikipedia

[building data: /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia]
19:43:42 | Downloading http://parl.ai/downloads/wizard_of_wikipedia/wizard_of_wikipedia.tgz to /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/wizard_of_wikipedia.tgz
19:44:27 | loading /usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/train.json
19:44:37 | Some data not being used. If you are not trying to reproduce the previous results, it is recommended that you run with the flag --add-missing-turns train or --add-missing-turns all.
- - - NEW EPISODE: wizard_of_wikipedia - - -
Science fiction
   I think science fiction is an amazing genre for anything. Future science, technology, time travel, FTL travel, they're all such interesting concepts.
I'm a huge fan of science fiction myself! 
   Awesome! I really love how sci-fi storytellers focus on political/social/philosophical issues that would still be around even in the future. Makes them relatable.
I agree. One of my favorit

In [3]:
with open("/usr/local/lib/python3.7/dist-packages/data/wizard_of_wikipedia/data.json") as f:
    lines = f.readlines()
lines[0]

In [4]:
import json
obj = json.loads(lines[0])

In [5]:
print(len(obj))
obj[0]

22311


{'chosen_topic': 'Science fiction',
 'chosen_topic_passage': ['Science fiction (often shortened to SF or sci-fi) is a genre of speculative fiction, typically dealing with imaginative concepts such as futuristic science and technology, space travel, time travel, faster than light travel, parallel universes, and extraterrestrial life.',
  'Science fiction often explores the potential consequences of scientific and other innovations, and has been called a "literature of ideas".',
  'It usually avoids the supernatural, unlike the related genre of fantasy.',
  'Historically, science-fiction stories have had a grounding in actual science, but now this is only expected of hard science fiction.',
  'Science fiction is difficult to define, as it includes a wide range of subgenres and themes.',
  'Hugo Gernsback, who suggested the term "scientifiction" for his "Amazing Stories" magazine, wrote: "By \'scientifiction\' I mean the Jules Verne, H. G. Wells and Edgar Allan Poe type of story—a charmin

In [6]:
print(len(obj))
len([a['text'] for b in obj for a in b['dialog']])

22311


201999

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install translate-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 6.9 MB/s 
     |████████████████████████████████| 76 kB 5.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14598 sha256=4e6a84b3d3342ac33b36d3c4594abae9230b12567820dc18b9006ff8c85dddcf
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built PyExecJS
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency 

In [9]:
import translators as ts
# print(ts._google.language_map)
import time
import os.path


def intTryParse(value):
    value = value.replace(" ","")
    value = value.replace("\u200b","")
    try:
        return int(value), True
    except ValueError:
        return value, False

lastest_dialog = -1
is_parsed = True
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow.txt"

if os.path.exists(output_path):
    with open(output_path) as f:
        lines = f.readlines()

    for i in range(1,len(lines)+1):
        line = lines[-i]
        if "<span class = 'notranslate'>" in line:
            lastest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
            if is_parsed: break
print(lastest_dialog)

tic = time.perf_counter()

if is_parsed or lastest_dialog == -1:
    text = ""
    # tic = time.perf_counter()
    for index, item in enumerate(obj):
        if index > lastest_dialog:
            item_text = '\n'.join( [a['text'] for a in item['dialog']])
            # print(index, "\n", item_text)

            text += f"\n<span class='notranslate'>{index}<span>\n{item_text}" 
            if (index + 1) % 3 == 0:
                translated = ts.google(text, from_language='en', to_language='fr')
                # print(translated)
                text = ""
                with open(output_path, "a") as f:
                    f.writelines("\n" + translated)
            
            if (index + 1) % 100 == 0:
                toc = time.perf_counter()
                print(f"{index + 1}\t{toc - tic:0.4f}\t", time.strftime("%H:%M:%S", time.gmtime()))
                tic = toc


    # print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")
else:
    print("Error")

Using United States server backend.


22310


In [10]:
# obj[308]
[a['text'] for a in obj[308]['dialog']]

['Do you think Computer science or the study of the theory, experimentation, and engineering is the most useful degree? ',
 'I definitely think it has a significant place in the modern age',
 'I think the study of  automating algorithmic processes is going to be to the next generation  what the business degree was to mine.',
 'Right, there will definitely be a great need for computer scientists',
 'I can only hope computer scientist also retain so of the common sense that you need in the real world. ',
 'Right, I guess the business students will still be needed in that case',
 'I guess we will always need people with finance, accounting, marketing or strategy degrees.',
 'Right, I wonder if we will see people moving away from those degrees towards computer science',
 'I think we all ready have, well Computer science and other degrees that focus on challenges in implementing computation.',
 'That makes sense!']

In [ ]:
# !cp french_wow.txt /content/drive/MyDrive/colabs/aliae-workspace/datasets/

Reading the generated file

In [11]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow.txt"
with open(output_path) as f:
    fr_lines = f.readlines()

In [12]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

In [13]:
len(fr_lines)

224310

In [27]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow_1.txt"
# len(fr_lines)
# display(fr_lines[:15])

dialog = []
all_dialogs = []

all_dialogs_in_parlai_format = []
number_of_odd_dialogs = 0
number_of_dialog = 0

flag = False
dialog_index =0
for index, line in enumerate(fr_lines[1:]):
    if "<span" in line:
        if not flag:
            all_dialogs.append(dialog)
            dialog = []
        else:
            all_dialogs.append([])
            dialog = []
            flag = False
        
        dialog_index +=1
        if "<span class='notranslate'>" in line: 
            flag = True
    else:
        dialog.append(line)
all_dialogs.append(dialog)

# print(len([1 for a in all_dialogs if a == []]))
print(len(all_dialogs))
text = ""

count = 0
previous_index = 22310
for index, dialog in enumerate(all_dialogs):
    if index >= previous_index:
        if dialog == []: 
            count +=1
    #         # lastest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
    #         # print(index, line.replace("\n",""), lastest_dialog, number_of_dialog)
            # print(index)    
            item_text = '\n'.join( [a['text'].replace("\n","") for a in obj[index]['dialog']])
            # print(index, item_text,"\n")
            text += f"\n<span class='notranslate'> {index}</span>\n{item_text}" 
            translated = ts.google(text, from_language='en', to_language='fr')
            # print(translated)
            all_dialogs[index] = "\n" + translated

            print(count, index)

            text = ""
            # break
        else:
            item_text = '\n'.join([a.replace("\n","") for a in dialog])
            text += f"\n<span class = 'notranslate'> {index} </span>\n{item_text}" 
            all_dialogs[index] = text
            text = ""
        
        if index % 1 == 0:
            with open(output_path, "a") as f:
                f.writelines(all_dialogs[previous_index:index+1])
                previous_index = index
#     # if (index + 1) % 100 == 0:
#     #     text = ""
#     #     with open(output_path, "a") as f:
#     #         if len(correct_lines) > 0:
#     #             f.writelines(correct_lines)
#     #             correct_lines = []
#     #         if not translated:
#     #             f.writelines("\n" + translated)

#     #     number_of_dialog +=1

# print("done!")



# for line in fr_lines[2:]:
#     if "notranslate" in line:
#         if len(dialog) % 2 == 1 : number_of_odd_dialogs +=1
#         all_dialogs_in_parlai_format.append(transfer_list_of_turns_to_dialog(dialog))
#         dialog = []
#     else:
#         dialog.append(line)
# # appending the last dialog
# all_dialogs_in_parlai_format.append(transfer_list_of_turns_to_dialog(dialog))

# print("done!", f"No. of all dialogs: {len(all_dialogs_in_parlai_format)}", "\t", f"No. of odd dialogs: {number_of_odd_dialogs}")

22311


In [47]:
output_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wow_1.txt"
with open(output_path) as f:
    fr_lines = f.readlines()
dialog_index = 0
for line in fr_lines:
    if "<span" in line:
        latest_dialog, is_parsed = intTryParse(line[line.find(">")+1:line.find("<",1)])
        if dialog_index != latest_dialog:
            print("break", dialog_index, latest_dialog)
            break
        dialog_index +=1
        # break
        # print(index, line.replace("\n",""), latest_dialog, number_of_dialog)
print("done!")

done!
